# Prerequisites

Required CSVS:

data/raw_food_data.csv 

data/ipc-raw.csv

data/data/PriceData_FEWS_NET.csv

In [171]:
import numpy as np
import pandas as pd
import pystan
import datetime as dt
import scipy.special

In [45]:
def strdate(s):
    date = dt.datetime.strptime(s, '%Y-%m-%d')
    year = date.date().year
    return year + (date.date().timetuple().tm_yday-1)/ (366 if year%4==0 else 365)

In [182]:
#Transform WFP Raw dataset to food dataset
def applyProductMap(s):
    return WANTEDPRODUCTS[str(s)]
def regionTransform(region):
    words = region.split(" ")
    if(len(words) == 2):
        if words[1] == 'Hoose':
            return 'Lower '+words[0]
        elif words[1] == 'Dhexe':
            return 'Middle '+words[0]
    return region



#Found by inspection of CSVs for 'best' market
market_map = {
    "Awdal": "Borama",
    "Bakool": "Xudur",
    "Banadir": "Mogadishu",
    "Bari":"Bossaso",
    "Bay": "Baidoa",
    "Galgaduud": "Dhuusamarreeb",
    "Gedo": "Doolow",
    "Hiraan": "Beletweyne",
    "Lower Juba": "Kismayo",
    "Lower Shabelle": "Qorioley",
    "Middle Juba": "Buale",
    "Middle Shabelle": "Balcad",
    "Mudug": "Gaalkacyo",
    "Nugaal": "Garowe",
    "Sanaag": "Ceerigaabo",
    "Sool": "Lasanod",
    "Togdheer": "Burco",
    "Woqooyi Galbeed":"Hargeysa"
}

def get_items(region):
    if(region == "Lower Shabelle"):
        return ["Maize (white)"]
    elif(region in {"Lower Juba", "Middle Juba"}):
        return ["Maize (white)", "Rice (imported)"]
    elif(region == "Sool"):
        return ["Rice (imported)", "Sorghum (white)"]
    elif(region == "Galgaduud"):
        return ["Maize (white)", "Sorghum (white)", "Rice (imported)"]
    else:
        return ["Maize (white)", "Sorghum (red)", "Rice (imported)"]

def fill_gaps(original):
    modified = [i for i in original]
    last_seen = np.nan
    gaps = []
    for i,val in enumerate(original):
        if(np.isnan(val)):
            gaps.append(i)
        else:
            if(len(gaps)!=0):
                if(np.isnan(last_seen)):
                    values = [val for _ in gaps]
                else:
                    values = np.linspace(last_seen, val, num=len(gaps)+2, endpoint=True).tolist()[1:-1]
                for (j,gap) in enumerate(gaps):
                    modified[gap] = values[j]
                gaps = []
            last_seen = val
    if(len(gaps)!=0):
        for gap in gaps:
            modified[gap] = last_seen
    return modified

MONTHS = list(np.cumsum([0, 31,28,31,30,31,30,31,31,30,31,30,31]))
LEAPS = list(np.cumsum([0, 31,29,31,30,31,30,31,31,30,31,30,31]))

def ym_to_date(years, months):
    dates = []
    for i in range(len(years)):
        y = years[i]
        m = months[i]-1
        dates.append(y + (LEAPS[m]/366 if y%4==0 else MONTHS[m]/365))
    return dates

STARTTIME = 2016

food_df = pd.read_csv('data/raw_food_data.csv')
food_df = food_df[food_df.Year.ge(STARTTIME-1)].rename(
    columns = {
        "Commodity":"Item", "Admin 1":"Region", "Price":"Raw_Price"
    })[['Region', 'Market', 'Item', 'Year','Month','Raw_Price']]
food_df.Region = food_df.Region.apply(regionTransform)
REGIONS = sorted(list(set(food_df.Region.values)))
new_df = pd.DataFrame(columns = ["Date","Region", "Market","Item","Price"])
for REGION in REGIONS:
    items = get_items(REGION)
    
    
    
    region_df = food_df[food_df.Market.eq(market_map[REGION]) & food_df.Item.isin(items)]
    
    
    min_y = min(region_df.Year.values)
    min_m = min(region_df[region_df.Year.eq(min_y)].Month.values)
    
    max_y = max(region_df.Year.values)
    max_m = max(region_df[region_df.Year.eq(max_y)].Month.values)
    
    for item in items:
        item_df = region_df[region_df.Item.eq(item)]
        
        
        item_min_y = min(item_df.Year.values)
        item_min_m = min(item_df[item_df.Year.eq(item_min_y)].Month.values)
        
        item_max_y = max(item_df.Year.values)
        item_max_m = max(item_df[item_df.Year.eq(item_max_y)].Month.values)
        
        new_rows = []
        if(item_min_m > min_m):
            for i in range(min_m, item_min_m):
                new_rows.append(dict(
                    Region = REGION,
                    Market = market_map[REGION],
                    Item = item,
                    Year = min_y,
                    Month = i,
                    Raw_Price = np.nan
                ))
        if(item_max_m < max_m):
            for i in range(item_max_m+1, max_m+1):
                new_rows.append(dict(
                    Region = REGION,
                    Market = market_map[REGION],
                    Item = item,
                    Year = max_y,
                    Month = i,
                    Raw_Price = np.nan
                ))
        if(len(new_rows)!=0):
            item_df = item_df.append(new_rows)
        item_df = item_df.sort_values(by=["Year", "Month"])
        
        new_prices = fill_gaps(item_df.Raw_Price.values.tolist())
        dates = ym_to_date(item_df.Year.values.tolist(), item_df.Month.values.tolist())
        item_df.insert(len(item_df.columns), "Price", new_prices)
        item_df.insert(len(item_df.columns), "Date", dates)
        new_df = new_df.append(item_df[new_df.columns], ignore_index=True)

new_df.to_csv("data/clean_food.csv", index=False)
display(new_df)

,Date,Region,Market,Item,Price
0,2009.000000,Awdal,Borama,Maize (white),5000.0
1,2009.084932,Awdal,Borama,Maize (white),5000.0
2,2009.161644,Awdal,Borama,Maize (white),3000.0
3,2009.246575,Awdal,Borama,Maize (white),3000.0
4,2009.328767,Awdal,Borama,Maize (white),2000.0
...,...,...,...,...,...
4723,2019.580822,Woqooyi Galbeed,Hargeysa,Rice (imported),4100.0
4724,2019.665753,Woqooyi Galbeed,Hargeysa,Rice (imported),4100.0
4725,2019.747945,Woqooyi Galbeed,Hargeysa,Rice (imported),4100.0
4726,2019.832877,Woqooyi Galbeed,Hargeysa,Rice (imported),4690.0


In [158]:
#Transform Raw IPC data to IPC dataset
MONTHS = list(np.cumsum([0, 31,28,31,30,31,30,31,31,30,31,30,31]))
LEAPS = list(np.cumsum([0, 31,29,31,30,31,30,31,31,30,31,30,31]))

ipc_raw_df = pd.read_csv('data/ipc-raw.csv')
regions = set(ipc_raw_df.Region.values)
seasons = set(ipc_raw_df.Season.values)
redundant_seasons = {'2017 (Apr-Jun)', '2019 (Jul-Sep)', '2019 (Apr-Jun)'}
seasons = seasons - redundant_seasons
season_time_map = dict()
for season in seasons:
    year = int(season[:4])
    last_month = season[-4:-1]
    time = None
    if(last_month == "Dec"):
        time = year + ( (LEAPS[9]/366) if year % 4 == 0 else (MONTHS[9]/365) )
    elif(last_month == "Jul"):
        time = year + ( (LEAPS[6]/366) if year % 4 == 0 else (MONTHS[6]/365) )
    elif(last_month == "Jun"):
        time = year + ( (LEAPS[3]/366) if year % 4 == 0 else (MONTHS[3]/365) )
    elif(last_month == "Jan"):
        time = year + 0.
    season_time_map[season] = time
    
ipc_rows = []
for season in seasons:
    ipc_season_df = ipc_raw_df[ipc_raw_df.Season.eq(season)]
    for region in regions:
        ipc_region_df = ipc_season_df[ipc_season_df.Region.eq(region)]
        time = season_time_map[season]
        total_pop = sum(map(lambda x: int(x.replace(',','') if (type(x)==type("String")) else 0), ipc_region_df['LZ pop..'].values))
        stressed = sum(map(lambda x: int(x.replace(',','')), ipc_region_df['Stressed\nIPC Phase 2'].values))
        crisis = sum(map(lambda x: int(x.replace(',','')), ipc_region_df['Crisis\nIPC Phase 3'].values))
        emergency = sum(map(lambda x: int(x.replace(',','')), ipc_region_df['Emergency\nIPC Phase 4'].values))
        catastrophe = sum(map(lambda x: int(x.replace(',','')), ipc_region_df['Catastrophe\nIPC Phase 5'].values))
        
        idp_region_df = ipc_region_df.loc[ipc_season_df['LZ Type'].eq('IDP')]
        idp_pop = sum(map(lambda x: int(x.replace(',','') if (type(x)==type("String")) else 0), idp_region_df['LZ pop..'].values))
        row = dict(
            time=time,
            region = region,
            pop = total_pop,
            idp_pop = idp_pop,
            p2count = stressed,
            p2perc = stressed/total_pop,
            p3count = crisis,
            p3perc = crisis/total_pop,
            p4count = emergency,
            p4perc = emergency/total_pop,
            p5count = catastrophe,
            p5perc = catastrophe/total_pop
        )
        ipc_rows.append(row)

ipc_df = pd.DataFrame(ipc_rows).sort_values(by='time')
ipc_df.to_csv('data/ipc-clean.csv', index=False)

In [181]:
#Transform FEWSNET Food data to FEWS dataset
MONTHS = list(np.cumsum([0, 31,28,31,30,31,30,31,31,30,31,30,31]))
LEAPS = list(np.cumsum([0, 31,29,31,30,31,30,31,31,30,31,30,31]))

def dateToFloat(d):
    #d gives last day of the month, we add one day to get start of next month
    year= int(d[6:8])+(2000 if int(d[6:8]) < 21 else 1900)
    mo = int(d[3:5]) - 1
    day = int(d[0:2])
    if(year%4==0):
        return year+(LEAPS[mo]+day)/366
    else:
        return year+(MONTHS[mo]+day)/365
    
mkt_to_region = {
    #'Abudwak':'Galgaduud',
    'Adanyabal': 'Middle Shabelle',
    #'Afgoi': 'Lower Shabelle',
    #'Afmadow':'Lower Juba',
    'Baidoa': 'Bay',
    #'Bardera': 'Gedo',
    #'Beled Hawa': 'Gedo',
    'Beled Weyn': 'Hiraan',
    'Borama': 'Awdal',
    'Bossaso': 'Bari',
    'Buale':'Middle Juba',
    'Burao':'Togdheer',
    #'Dinsor':'Bay',
    #'El Barde': "Bakool",
    #'El Dhere': 'Galgaduud',
    #'Elwak': 'Gedo',
    'Erigavo': 'Sanaag',
    'Gadon, Dhusamareb': 'Galgaduud',
    'Galkayo': 'Mudug',
    'Garowe': 'Nugaal',
    #'Hara Dhere': 'Mudug',
    'Hargeisa': 'Woqooyi Galbeed',
    'Hudur': 'Bakool',
    #'Jamame': 'Lower Juba',
    #'Jilib': 'Middle Juba',
    #'Jowhar': 'Middle Shabelle',
    'Kismayo': 'Lower Juba',
    'Lasanod': 'Sool',
    'Lugh': 'Gedo',
    #'Merka': 'Lower Shabelle',
    'Mogadishu, Bakara': 'Banadir',
    'Qorioley': 'Lower Shabelle',
    #'Wanle Weyne': 'Lower Shabelle',
    #'Zeilac/Lawayacado': 'Awdal'
}

market_rename={
    "Hudur": "Xudur",
    "Mogadishu, Bakara": "Mogadishu",
    "Gadon, Dhusamareb": "Dhuusamarreeb",
    "Beled Weyn": "Beletweyne",
    "Galkayo":"Gaalkacyo",
    "Erigavo":"Ceerigaabo",
    "Burao":"Burco",
    "Hargeisa": "Hargeysa"
}

market_map = {
    "Awdal": "Borama",
    "Bakool": "Xudur",
    "Banadir": "Mogadishu",
    "Bari":"Bossaso",
    "Bay": "Baidoa",
    "Galgaduud": "Dhuusamarreeb",
    "Gedo": "Lugh",
    "Hiraan": "Beletweyne",
    "Lower Juba": "Kismayo",
    "Lower Shabelle": "Qorioley",
    "Middle Juba": "Buale",
    "Middle Shabelle": "Adanyabal",
    "Mudug": "Gaalkacyo",
    "Nugaal": "Garowe",
    "Sanaag": "Ceerigaabo",
    "Sool": "Lasanod",
    "Togdheer": "Burco",
    "Woqooyi Galbeed":"Hargeysa"
}
def get_region(mkt):
    if(mkt in mkt_to_region):
        return mkt_to_region[mkt]
    else:
        return ''

def rename_market(mkt):
    if(mkt in market_rename):
        return market_rename[mkt]
    else:
        return mkt
    
rows=[]
food_df = pd.read_csv('data/PriceData_FEWS_NET.csv', usecols = [0,1,2,3,5,7,12])
food_df['Date'] = food_df['period_date'].apply(dateToFloat)
food_df = food_df.loc[food_df.country.eq("Somalia") & food_df.Date.ge(2015) & food_df.product_source.eq('Local')]
food_df['Region'] = food_df['market'].apply(get_region)
food_df = food_df[food_df.Region.ne('')]
food_df['Market'] = food_df['market'].apply(rename_market)
food_df = food_df.rename(columns = {
    "product":"Item",
    "value":"Raw_Price"
})[['Date','Region','Market','Item','Raw_Price']]

regions = set(food_df.Region.values)
items = set()
for region in regions:
    avail_items = set(food_df[food_df.Region.eq(region)].Item.values)
    if(len(items) == 0):
        items = avail_items
    items = items.intersection(avail_items)
food_df = food_df[food_df.Item.isin(items)]

def fill_gaps(original):
    modified = [i for i in original]
    last_seen = np.nan
    gaps = []
    for i,val in enumerate(original):
        if(np.isnan(val)):
            gaps.append(i)
        else:
            if(len(gaps)!=0):
                if(np.isnan(last_seen)):
                    values = [val for _ in gaps]
                else:
                    values = np.linspace(last_seen, val, num=len(gaps)+2, endpoint=True).tolist()[1:-1]
                for (j,gap) in enumerate(gaps):
                    modified[gap] = values[j]
                gaps = []
            last_seen = val
    if(len(gaps)!=0):
        for gap in gaps:
            modified[gap] = last_seen
    return modified


new_df = pd.DataFrame(columns = ["Date","Region", "Market","Item","Price"])
regions = sorted(list(regions))

for region in regions:
    display(set(food_df[food_df.Region.eq(region)].Market.values))
    if(region != "Sanaag"):
        continue
    for item in items:
        if(item != "Cowpeas (Red)"):
            continue
        market_df = food_df.loc[food_df.Region.eq(region) & food_df.Market.eq(market_map[region]) & food_df.Item.eq(item)]
        display(market_df)
        
        new_prices=fill_gaps(market_df.Raw_Price.tolist())
        market_df.insert(len(market_df.columns), "Price", new_prices)
        new_market_df = market_df.rename(columns = {
            "product":"Item",
        })[['Date','Region','Market','Item','Price']]
        display(new_market_df)
        new_df = new_df.append(new_market_df, ignore_index = True)
new_df.to_csv('data/clean_fews.csv', index=False)

{'Borama'}

{'Xudur'}

{'Mogadishu'}

{'Bossaso'}

{'Baidoa'}

...

,Date,Region,Market,Item,Raw_Price
463741,2015.413699,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463742,2015.495890,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463743,2015.580822,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463744,2015.665753,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463745,2015.747945,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463746,2015.832877,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463747,2015.915068,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463748,2016.000000,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463749,2016.084699,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463750,2016.163934,Sanaag,Ceerigaabo,Cowpeas (Red),NaN


,Date,Region,Market,Item,Price
463741,2015.413699,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463742,2015.495890,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463743,2015.580822,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463744,2015.665753,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463745,2015.747945,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463746,2015.832877,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463747,2015.915068,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463748,2016.000000,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463749,2016.084699,Sanaag,Ceerigaabo,Cowpeas (Red),NaN
463750,2016.163934,Sanaag,Ceerigaabo,Cowpeas (Red),NaN


{'Lasanod'}

{'Burco'}

{'Hargeysa'}